# Single neuron using Flux.jl

## Read in and process data

In [13]:
using CSV

apples = CSV.read("Apple_Golden_1.dat", delim='\t')
bananas = CSV.read("bananas.dat", delim='\t');

In [14]:
col1 = :red
col2 = :green

x_apples  = [ [apples[i, col1], apples[i, col2]] for i in 1:size(apples)[1] ]
x_bananas = [ [bananas[i, col1], bananas[i, col2]] for i in 1:size(bananas)[1] ]

xs = vcat(x_apples, x_bananas)

ys = vcat( zeros(size(x_apples)[1]), ones(size(x_bananas)[1]) );

The input data is in `xs` and the labels in `y`

## Using Flux.jl

In [6]:
using Flux

INFO: Recompiling stale cache file /Users/dpsanders/.julia/lib/v0.6/Flux.ji for module Flux.


The function $\sigma$ that we have been using is predefined by Flux:

In [8]:
σ

σ (generic function with 3 methods)

In [9]:
methods(σ)

# 3 methods for generic function "σ":
σ(x::Float32) in NNlib at /Users/dpsanders/.julia/v0.6/NNlib/src/activation.jl:32
σ(x::ForwardDiff.Dual{T,Float32,N} where N) where T in NNlib at /Users/dpsanders/.julia/v0.6/NNlib/src/activation.jl:36
σ(x) in NNlib at /Users/dpsanders/.julia/v0.6/NNlib/src/activation.jl:25

In [10]:
?σ

"σ" can be typed by \sigma<tab>

search: σ



```
σ(x) = 1 / (1 + exp(-x))
```

Classic [sigmoid](https://en.wikipedia.org/wiki/Sigmoid_function) activation function.

```
1 │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡆⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢀⣀⣀│
  │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢀⡠⠔⠒⠉⠉⠀⠀│
  │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡇⠀⠀⠀⠀⠀⠀⠀⠀⣀⠤⠚⠁⠀⠀⠀⠀⠀⠀⠀│
  │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡇⠀⠀⠀⠀⠀⠀⡤⠊⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│
  │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡇⠀⠀⠀⢀⡔⠉⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│
  │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡇⠀⠀⡔⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│
  │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡇⡔⠊⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│
  │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡠⡏⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│
  │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡠⠜⠀⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│
  │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢀⠜⠀⠀⠀⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│
  │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣀⠜⠁⠀⠀⠀⠀⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│
  │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢀⡠⠚⠀⠀⠀⠀⠀⠀⠀⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│
  │⠀⠀⠀⠀⠀⠀⠀⢀⡤⠒⠉⠀⠀⠀⠀⠀⠀⠀⠀⠀⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│
  │⠀⠀⣀⣀⠤⠔⠊⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│
0 │⠋⠉⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│
  -3                      0                      3
```


We can make a neuron in a simple way:

In [14]:
model = Dense(2, 1, σ)

Dense(2, 1, NNlib.σ)

In [15]:
typeof(model)

Flux.Dense{NNlib.#σ,TrackedArray{…,Array{Float64,2}},TrackedArray{…,Array{Float64,1}}}

We have made an object of type `Dense`, defined by `Flux`. This represents a "dense neural network layer" (see later).
Inside the object live the parameters that we will modify during the learning process:

In [16]:
model.W

Tracked 1×2 Array{Float64,2}:
 1.12843  0.788773

In [17]:
model.b

Tracked 1-element Array{Float64,1}:
 0.0

The fact that `W` and `b` are of size $1 \times 2$ and $1$, respectively, comes from the `(2, 1)` pair in the call to the `Dense` constructor when we created `model`. A "tracked" array is a special type provided by `Flux.jl` that is able to calculate ("track") derivatives via reverse-mode automatic differentiation, usually called **backpropagation** in the context of neural networks. This is more efficient in this context than calculating the derivatives via forward-mode automatic differentiation, as we did previously using the `ForwardDiff.jl` package.

## 

In [53]:
W = rand(1, 2)
b = rand(1)

predict(x) = σ.(W*x + b)
loss(x, y) = sum(abs2, (predict(x) .- y) )

x, y = rand(2), rand(1) 
loss(x, y) 

0.08786885598447765

We will now see how `Flux.jl` facilitates the type of calculations that we have been doing.
To do so, we use the `param` function to define objects that will contain both the values of 
the parameters `W` and `b` *and* the derivatives. These derivatives will be the derivatives of the loss 
function with respect to `W` and `b` that we calculated previously using `ForwardDiff`.

Let's start, as usual, by setting up some random initial values for the parameters:

In [61]:
W_data = rand(1, 2)  
b_data = rand(1)

W_data, b_data

([0.718982 0.414688], [0.692976])

We now set up `Flux.jl` objects that will contain these values *and* their derivatives, and allow to propagate
this information around:

In [62]:
W = param( W_data )
b = param( b_data )

predict(x) = σ.(W*x + b)
loss(x, y) = sum( (predict(x) .- y).^2 )

x, y = rand(2), rand(1) 
l = loss(x, y) 

Tracked 0-dimensional Array{Float64,0}:
0.0443256

In [63]:
fieldnames(W)

4-element Array{Symbol,1}:
 :ref 
 :f   
 :data
 :grad

We see that the data is indeed inside the object:

In [64]:
W.data  # the random 

1×2 Array{Float64,2}:
 0.718982  0.414688

Initially, the derivatives are zero:

In [57]:
W.grad

1×2 Array{Float64,2}:
 0.0  0.0

Having set up the structure, we can now propagate the derivative information backwards 
from the `loss` function to all of the objects that are used to calculate it:

In [65]:
using Flux.Tracker

back!(l)   # backpropagate derivatives of the loss function

In [66]:
W.grad

1×2 Array{Float64,2}:
 -0.00819256  -0.0500179

In [67]:
b.grad

1-element Array{Float64,1}:
 -0.0821254

We can now use this structure to do stochastic gradient descent, just as we did in the previous notebook.

**Exercise:** Implement this.

In [76]:
function stochastic_gradient_descent(loss, xs, ys, W, b, N=1000)

    η = 0.01

    for i in 1:N
        
        which = rand(1:length(xs))  # choose a data point
        
        xx = xs[which]
        yy = ys[which]
        
        l = loss(xx, yy)
        back!(l)
        
        W.data -= η * W.grad
        b.data -= η * b.grad
    
    end
    
    return W, b
    
end
    

stochastic_gradient_descent (generic function with 2 methods)

In [77]:
b

Tracked 1-element Array{Float64,1}:
 0.692976

In [78]:
ys

982-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮  
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0

In [82]:
W_final, b_final = stochastic_gradient_descent(loss, xs, ys, W, b)

(param([-70.0661 -72.9035]), param([87.9345]))

In [83]:
W_final

Tracked 1×2 Array{Float64,2}:
 -70.0661  -72.9035

In [84]:
b_final

Tracked 1-element Array{Float64,1}:
 87.9345

In [80]:
using Plots; gr()

Plots.GRBackend()

In [81]:
scatter(first.(x_apples), last.(x_apples), m=:cross)
scatter!(first.(x_bananas), last.(x_bananas))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.50 
 
 
 0.55 
 
 
 0.60 
 
 
 0.65 
 
 
 0.70 
 
 
 0.40 
 
 
 0.45 
 
 
 0.50 
 
 
 0.55 
 
 
 0.60 
 
 
 0.65 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<line clip-path="url(#clip7702)" x1="532.999

Let's draw the function that the network has learned, together with the data:

In [89]:
heatmap(0:0.01:1, 0:0.01:1, (x,y)->predict([x, y]).data[1])

scatter!(first.(x_apples), last.(x_apples), m=:cross)
scatter!(first.(x_bananas), last.(x_bananas))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.00 
 
 
 0.25 
 
 
 0.50 
 
 
 0.75 
 
 
 1.00 
 
 
 0.00 
 
 
 0.25 
 
 
 0.50 
 
 
 0.75 
 
 
 1.00 
 
 
<image width="454" height="343" xlink:href="data:image/png;base64,
iVBORw0KGgoAAAANSUhEUgAAAcYAAAFXCAYAAAA4WO7PAAAbBElEQVR4nO3dWXcdx5Eg4KwqXICk
SFFctFCStUsUJUu2errdm932j50fMmfOPMyxrc22vLZsy7ZEgoso7gTuBTAPPd2dGZE2IQggCfD7
3rJOoOoCWuJmZEbWsNj6n1tlv9nWJ85BW+Ha1tZGjtmcN+ONjVspZnH3YjMer/42xax8+mF7nx9f
STG//z/vNeP/9ekbKeZ/r64044/nX6SY85vt82/PL6WYjc3299jcmqeYUjbb4VbvD731N0YA+9/4
oD8AADxMJEYAqEiMAFBZetAfYEeGzrW02JWD0pVhyjHpq8Jj+VmHnmqGi5P5WWvhRsvjhynmlfH9
ZjyOmylmGs62MavPpZhhvX3WF7P8eW7P23XRsnk7xWyFdcetIa/Bxr/z0F3LBdi/zBgBoCIxAkBl
f5ZSe2L1sFvPG/7G6P//WCivjt2vDqG8upLvtDjR/uD6q/lGsbz68vRBionl1Wl4M8esnmnHa/lZ
n4fy6q1YWi25paP3N0zl1U5MLK8qrQL7iRkjAFQkRgCoSIwAUDk4a4zRLrV0xDXHUnrrjkfys1ZO
N8PFiV5LR3ttpbOg+eLYrjv+qNPSMQ7n2vHq0znmbvusvyznz3NzvtqMe8fhxb+hlg7goDFjBICK
xAgAlYNbSu3Zs5aOTtQQ75Nj5k+019Zeyt9TYnn1hc4JOrG8Og1vpZjxQntaz3AnP+vPs/barXIh
xSxieXVrPcVo6QD2MzNGAKhIjABQebRKqdGu7VzN3y/GMmsvLOWdq7NQbo2l1VI6O1c7u2SfHz9q
xj+cejtX327G04XTOeZ2W4L9U+cw8puhvLrYuJliYnnVzlVgPzFjBICKxAgAFYkRACqP9hpjz661
dLTfOdKaYymlhOXC2cqpFJJaOjrrmcuhpePZ6Wcp5oeppePbKWa8cLId33o7xXwW1h1vdFs6brQX
tHQA+4gZIwBUJEYAqCil3sv9bOnInRhlttyWV+fH87PWQ0tHLK2WUsqZ8RfN+Aedw8in4Z328114
IsWMN9vy6h+W87Nurn/RjOc7buloP6OWDuB+MGMEgIrECAAViREAKtYYd+I+tnQMU/jJ5ZMpJq47
rvdaOsK1Z6ZPUsz3w1FyY1hzLKWU6cKJNuZ6bun4fXgJ8o2w5lhKKfNttXTEC3ldVEsHsNvMGAGg
IjECQEUpdTf06qRRqgtur6VjCP+IpunwPT/A4vFOS8fz7b1nY+4NeWpsT8z5fq+lY3y3GY9DLu0O
obwaS6ullHJ9Hls6rqeYshlbOnKIlg5gt5kxAkBFYgSAilLq/dLflrqNkDHE5H9k03QoXDmRYmJ5
dd4p25Zw7XR4AXIppfzLFA8jfzfFTEP7EuTxWt65+umsfdb18nmKWS+hvLq5lmLsXAV2mxkjAFQk
RgCoSIwAULHG+CDlfo1thGynpSOuOZZShvZNGYvHX04h8/CWjtJ5S8fp6YNm/M/dt3R8p73N8GSK
Gb96qxn/btZp6QjrjuvlWoqJ645aOoBvyowRACoSIwBUlFIfJjtu6ZhCTI6axnCfTulyfqwtr/Za
OmIp9+T4for5x9jS8eF3Usw0PN2Mx6tvpZjfhpcgX8/njKfy6qaWDuAbMmMEgIrECAAViREAKtYY
H3Y7aOmIR7v9h/Yf9Tjmlo5ZeE/y4uiLKWZ+Jty709JxIqw7fq/T0jHGlo7yTI65eq4Z/6bzlo5r
639uxr2WjrjuuDUsUoyWDuA/mTECQEViBICKUup+0+3XuHfQkMqr+R99LK8udVo6FkfjWzo6Hyg8
63inpeMfpo1mPI3vpZhpONN+vi/PpZjfhJaOq/M/pZj1xVfNeLNTE90qobyqpQMeWWaMAFCRGAGg
opR6EKSdq/cOyqXVUvLO1U7E7HgzjqXVUjon5nSe9fjUllf/x5TLrdPwXhg/l2LGK28241/P8rOu
ls+a8driaoqJhdNUWi3FzlV4RJgxAkBFYgSAisQIABVrjAfRnrZ0tD+3tNRp6XgstHQ83XlLR1jA
PDb+NMW8F9o8pu4JOs+34ytnU8yvQtvJl53fPa47bpa7KUZLBzwazBgBoCIxAkBFKfVRsVstHeE+
Y1lJIUtLjzfjWFotpZRFPDGn86yjobz63U5LxziElyIPL+SYy2159Zedlo5YXr27+DLFbJZwGPmO
WzoUWOFhZsYIABWJEQAqEiMAVKwxPqp22tIRv0t1vlqNQ7vu2PuXbONIe7zb4snOB3q7vfmRTkvH
u9to6ZiG9oXL4+XXU8wnYd3xSuetIXfn7brjtlo60mFzpQzh72zNER4uZowAUJEYAaCilMp/20lL
R++7VWzpmHJLR4mlym5LR3vvrU5580gopX6719Ixtr/IOLyUYy6/1ow/Wcq/16XY0jG/kmJieXVr
q9PSEVpMYmm1FOVVeJDMGAGgIjECQEUplb9ut3audu4zjsttyNKxHHSk/cGNzs7VtVBuPTzmUurb
U7ubtb9z9eX28116NcX8IpRXY2m1lFLuhPLqZrmTYlJ5dbBzFR4mZowAUJEYAaAiMQJAxRojX88e
tXSUsOZYSilLS0fbC4fPpJiN0+2945pjKaWshHXHc50TdPJbOl7JMZfadcifd1o6Lqa3dFxOMRub
7bqjlg54uJgxAkBFYgSAilIq38yutXT0ejraa0tLnZjDoaXjdI5ZD+XV5U5Lx5uhpWPstnS0LRzT
xZdSzM9CeXW183vdmbfl1VhaLaWUra15e2EbLR29urZiK3x9ZowAUJEYAaAiMQJAxRoju29HLR29
kKkdb6OlYzjUeUvHqfZav6Xjg2Z8ttfSEdYdxyG/8Hi8+EIz/rjz3XM1/La355dSzGZYUtyMa46l
5HXHzt95CBetOcK9mTECQEViBICKUip7b8ctHfHClGJieXXq/Rt9+OlmGEurpZSyFtoqVsYPU8zr
4SXI/ZaON9qYi8+nmI+32medn+XPk8qrm7dTTCqvdlo64t85llY7IfDIM2MEgIrECAAVpVQejD3a
uTp0dq6mn+ztXD25ncPI2/Lqa50TdMZwHE0srZaSy6vDen5WLK/2dq6WzVvNsHcY+dawES7k29i5
Ci0zRgCoSIwAUJEYAaBijZGHwy61dGx1WjrG9PXvsfysQ+1wcbLX0hHf0pFbOl4Zt9PScbaNWX02
xXwU1h0/77Z0XGzGG52Wjvg3TGuOnRgtHTzqzBgBoCIxAkBFKZWH1y61dMTyai6tlpLKqyudlo4T
7bX1zkuIl8ePmvHLvZaOKR5G/maOWX2mGQ9r+VmfL7e/yK311RSzEVo6en9DLR3QMmMEgIrECAAV
iREAKtYY2T8edEtHWHeMa46l5JaOlc7Rci9O7UuRf9Rt6TjXfr7Vp1PMeKe991+W8+e5NW/XHRcb
N1OMlg5omTECQEViBICKUir72561dHSihiPhPqdTyPyJ9ufWXs73WQl12xfGD1LMD0N5dRreSjHj
6pPt+Hb+nvuncGLOrXIhxSw2YkvHeorR0sGjxIwRACoSIwBUlFI5WHZt52r+zjiWWXth6UiKmYXT
cGJptZTeYeT5Wc+HE3T+rbNzdRzebscXTuWY220J9rPOYeQ3Q3m1v3O1La/aucpBZsYIABWJEQAq
EiMAVK

TODO: Animation of learning process

## Automation with Flux.jl

We will need to repeat the above process for a lot of different systems.
Fortunately, Flux.jl provides us with tools to automate this.

Firstly, we create the model:

In [15]:
using Flux

In [16]:
model = Dense(2, 1, σ)

Dense(2, 1, NNlib.σ)

In [17]:
model.W

Tracked 1×2 Array{Float64,2}:
 -0.466978  0.775266

In [18]:
model.b

Tracked 1-element Array{Float64,1}:
 0.0

We can use the `model` object just like a function to apply it to data:

In [19]:
model(rand(2))

Tracked 1-element Array{Float64,1}:
 0.555819

Flux has various loss functions built in:

In [20]:
loss(x, y) = Flux.mse(model(x), y)

loss (generic function with 1 method)

In [21]:
data = zip(xs, ys)

Base.Iterators.Zip2{Array{Array{Float64,1},1},Array{Float64,1}}(Array{Float64,1}[[0.708703, 0.641282], [0.648376, 0.553169], [0.647237, 0.553302], [0.647963, 0.55323], [0.647653, 0.554047], [0.648491, 0.553821], [0.647974, 0.554518], [0.649307, 0.554399], [0.648141, 0.554708], [0.64984, 0.555665]  …  [0.524028, 0.452379], [0.523906, 0.452571], [0.523823, 0.4514], [0.522489, 0.449973], [0.517573, 0.444391], [0.515956, 0.441912], [0.517585, 0.444827], [0.510357, 0.436022], [0.508873, 0.43433], [0.528205, 0.440139]], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0])

In [22]:
collect(data)

982-element Array{Tuple{Array{Float64,1},Float64},1}:
 ([0.708703, 0.641282], 0.0)
 ([0.648376, 0.553169], 0.0)
 ([0.647237, 0.553302], 0.0)
 ([0.647963, 0.55323], 0.0) 
 ([0.647653, 0.554047], 0.0)
 ([0.648491, 0.553821], 0.0)
 ([0.647974, 0.554518], 0.0)
 ([0.649307, 0.554399], 0.0)
 ([0.648141, 0.554708], 0.0)
 ([0.64984, 0.555665], 0.0) 
 ([0.648446, 0.555576], 0.0)
 ([0.709808, 0.632473], 0.0)
 ([0.650164, 0.555766], 0.0)
 ⋮                          
 ([0.52913, 0.44031], 1.0)  
 ([0.528731, 0.456548], 1.0)
 ([0.524028, 0.452379], 1.0)
 ([0.523906, 0.452571], 1.0)
 ([0.523823, 0.4514], 1.0)  
 ([0.522489, 0.449973], 1.0)
 ([0.517573, 0.444391], 1.0)
 ([0.515956, 0.441912], 1.0)
 ([0.517585, 0.444827], 1.0)
 ([0.510357, 0.436022], 1.0)
 ([0.508873, 0.43433], 1.0) 
 ([0.528205, 0.440139], 1.0)

In [27]:
opt = SGD([model.W, model.b], 0.01)
# give a list of the parameters that will be modified

(::#71) (generic function with 1 method)

In [28]:
for i in 1:100
    Flux.train!(loss, data, opt)
end

In [29]:
model.W

Tracked 1×2 Array{Float64,2}:
 -5.98412  -5.17245

In [30]:
model.b

Tracked 1-element Array{Float64,1}:
 7.00206

In [31]:
opt = SGD(params(model), 0.01)

(::#71) (generic function with 1 method)

In [32]:
params(model)

2-element Array{Any,1}:
 param([-5.98412 -5.17245])
 param([7.00206])          